___
## <center><a id=0 style='color:red'>Импорт библиотек</a>

In [1]:
from ds_nms import utils_io, feature_select
import pandas as pd
import numpy as np
import pickle

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.base import BaseEstimator
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.model_selection import (
    train_test_split,
    KFold,
    cross_validate,
    StratifiedGroupKFold,
    StratifiedKFold,
)
from sklearn.metrics import (
    mean_absolute_percentage_error,
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    median_absolute_error,
)
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    QuantileTransformer,
    Normalizer,
    MinMaxScaler,
)
from sklearn.decomposition import PCA
from sklearn.ensemble import (
    RandomForestRegressor,
    HistGradientBoostingRegressor,
    IsolationForest,
    GradientBoostingRegressor,
)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.feature_selection import RFE, SequentialFeatureSelector

np.set_printoptions(suppress=True, precision=2)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

d:\MTP\test2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


___
## <center><a id=1 style='color:red'>Импорт датасета</a>
 [К описанию](#99)

In [4]:
DIR_train = 'train_poly'

train_poly_data = utils_io.load_data( sub_dir=DIR_train )
X_train_poly_1 = train_poly_data['X_train_poly_1']
X_train_poly_2 = train_poly_data['X_train_poly_2']
X_train_poly_3 = train_poly_data['X_train_poly_3']
target_train_1 = train_poly_data['target_train_1']
target_train_2 = train_poly_data['target_train_2']
target_train_3 = train_poly_data['target_train_3']

Файл data/train_poly/target_train_1.pkl успешно загружен.
Файл data/train_poly/target_train_2.pkl успешно загружен.
Файл data/train_poly/target_train_3.pkl успешно загружен.
Файл data/train_poly/X_train_poly_1.pkl успешно загружен.
Файл data/train_poly/X_train_poly_2.pkl успешно загружен.
Файл data/train_poly/X_train_poly_3.pkl успешно загружен.


In [5]:
DIR_test = 'test_poly/'

test_poly_data = utils_io.load_data( sub_dir=DIR_test )
X_test_poly_1 = test_poly_data['X_test_poly_1']
X_test_poly_2 = test_poly_data['X_test_poly_2']
X_test_poly_3 = test_poly_data['X_test_poly_3']
target_test_1 = test_poly_data['target_test_1']
target_test_2 = test_poly_data['target_test_2']
target_test_3 = test_poly_data['target_test_3']

Файл data/test_poly//target_test_1.pkl успешно загружен.
Файл data/test_poly//target_test_2.pkl успешно загружен.
Файл data/test_poly//target_test_3.pkl успешно загружен.
Файл data/test_poly//X_test_poly_1.pkl успешно загружен.
Файл data/test_poly//X_test_poly_2.pkl успешно загружен.
Файл data/test_poly//X_test_poly_3.pkl успешно загружен.


___
___
## <center><a id=3 style='color:red'>Выбор факторов</a>

In [4]:
scoring=['neg_root_mean_squared_error']

feature_select.get_best_n_features(X=X_train_poly_1,
                                    y=target_train_1,
                                    selector=RFE,
                                    scoring=scoring,
                                    estimator=LinearRegression(),
                                    X_name='X_train_1',
                                    n_folds=4)

[I 2025-02-05 14:37:45,927] A new study created in memory with name: X_train_1_LinearRegression()_study
[I 2025-02-05 14:37:45,961] Trial 0 finished with value: 13657.819838184383 and parameters: {'n_features_to_select': 12}. Best is trial 0 with value: 13657.819838184383.
[I 2025-02-05 14:37:46,007] Trial 1 finished with value: 46175.77276696207 and parameters: {'n_features_to_select': 21}. Best is trial 0 with value: 13657.819838184383.
[I 2025-02-05 14:37:46,032] Trial 2 finished with value: 112561.26393671324 and parameters: {'n_features_to_select': 26}. Best is trial 0 with value: 13657.819838184383.
[I 2025-02-05 14:37:46,063] Trial 3 finished with value: 21354.588417827643 and parameters: {'n_features_to_select': 18}. Best is trial 0 with value: 13657.819838184383.
[I 2025-02-05 14:37:46,092] Trial 4 finished with value: 13205.466077573745 and parameters: {'n_features_to_select': 3}. Best is trial 4 with value: 13205.466077573745.
[I 2025-02-05 14:37:46,109] Trial 5 finished wit

(6, <optuna.study.study.Study at 0x118bdd61b80>)

In [ ]:
feature_select.get_best_n_features(X=X_train_poly_1,
                                    y=target_train_1,
                                    selector=SequentialFeatureSelector,
                                    scoring=scoring,
                                    estimator=LinearRegression(),
                                    X_name='X_train_1',
                                    n_folds=4,
                                    direction='backward')

In [ ]:
feature_select.get_best_n_features(X=X_train_poly_1,
                                    y=target_train_1,
                                    selector=SequentialFeatureSelector,
                                    scoring=scoring,
                                    estimator=LinearRegression(),
                                    X_name='X_train_1',
                                    n_folds=4,
                                    direction='forward')

___
### <a id=5>Recursive feature elimination</a>

In [6]:
RAND_STATE = 1
scoring=['neg_root_mean_squared_error']
estimators_lst = [
    LinearRegression(),
    DecisionTreeRegressor(random_state=RAND_STATE),
                ]

data_dict = {
    "X_train_fs_1": (X_train_poly_1, target_train_1),
    "X_train_fs_2": (X_train_poly_2, target_train_2),
    "X_train_fs_3": (X_train_poly_3, target_train_3),
        }

___

In [7]:
result_df = feature_select.save_selected_features(data_dict=data_dict,
                                    selector=RFE,
                                    estimators=estimators_lst,
                                    scoring=scoring,
                                    output_dir=DIR_train)

result_df

Файл data/train_poly\f_select_2025_02_05_14_43\X_train_fs_1_LinearRegression.pkl успешно сохранён.
Файл data/train_poly\f_select_2025_02_05_14_43\X_train_fs_2_LinearRegression.pkl успешно сохранён.
Файл data/train_poly\f_select_2025_02_05_14_43\X_train_fs_3_LinearRegression.pkl успешно сохранён.
Файл data/train_poly\f_select_2025_02_05_14_43\X_train_fs_1_DecisionTreeRegressor.pkl успешно сохранён.
Файл data/train_poly\f_select_2025_02_05_14_43\X_train_fs_2_DecisionTreeRegressor.pkl успешно сохранён.
Файл data/train_poly\f_select_2025_02_05_14_43\X_train_fs_3_DecisionTreeRegressor.pkl успешно сохранён.


,number,value,datetime_start,datetime_complete,duration,params_n_features_to_select,system_attrs_grid_id,system_attrs_search_space,state,data_name,estimator
0,0,13657.819838,2025-02-05 14:43:13.572825,2025-02-05 14:43:13.598819,0 days 00:00:00.025994,12,0,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_fs_1,LinearRegression
1,1,46175.772767,2025-02-05 14:43:13.599826,2025-02-05 14:43:13.620825,0 days 00:00:00.020999,21,1,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_fs_1,LinearRegression
2,2,112561.263937,2025-02-05 14:43:13.620825,2025-02-05 14:43:13.638818,0 days 00:00:00.017993,26,2,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_fs_1,LinearRegression
3,3,21354.588418,2025-02-05 14:43:13.639824,2025-02-05 14:43:13.677818,0 days 00:00:00.037994,18,3,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_fs_1,LinearRegression
4,4,13205.466078,2025-02-05 14:43:13.679818,2025-02-05 14:43:13.718819,0 days 00:00:00.039001,3,4,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_fs_1,LinearRegression
...,...,...,...,...,...,...,...,...,...,...,...
199,29,2008.530630,2025-02-05 14:43:20.955825,2025-02-05 14:43:21.015824,0 days 00:00:00.059999,8,29,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_fs_3,DecisionTreeRegressor
200,30,2252.711587,2025-02-05 14:43:21.016818,2025-02-05 14:43:21.040819,0 days 00:00:00.024001,33,30,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_fs_3,DecisionTreeRegressor
201,31,2082.540968,2025-02-05 14:43:21.041818,2025-02-05 14:43:21.075826,0 days 00:00:00.034008,28,31,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_fs_3,DecisionTreeRegressor
202,32,1976.822355,2025-02-05 14:43:21.076817,2025-02-05 14:43:21.142820,0 days 00:00:00.066003,4,32,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_fs_3,DecisionTreeRegressor


In [14]:
min_result_df_1 = result_df[result_df['data_name'] == 'X_train_fs_1' ]
min_result_df_1 = min_result_df_1[min_result_df_1['value'] == min_result_df_1['value'].min()]
min_result_df_1

,number,value,datetime_start,datetime_complete,duration,params_n_features_to_select,system_attrs_grid_id,system_attrs_search_space,state,data_name,estimator


In [8]:
min_result_df_2 = result_df[result_df['data_name'] == 'X_train_fs_2' ]
min_result_df_2 = min_result_df_2[min_result_df_2['value'] == min_result_df_2['value'].min()]
min_result_df_2

,number,value,datetime_start,datetime_complete,duration,params_n_features_to_select,system_attrs_grid_id,system_attrs_search_space,state,data_name,estimator
63,29,1492.854428,2025-02-05 14:35:21.180308,2025-02-05 14:35:21.206306,0 days 00:00:00.025998,8,29,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_fs_2,LinearRegression


In [9]:
min_result_df_3 = result_df[result_df['data_name'] == 'X_train_fs_3' ]
min_result_df_3 = min_result_df_3[min_result_df_3['value'] == min_result_df_3['value'].min()]
min_result_df_3


,number,value,datetime_start,datetime_complete,duration,params_n_features_to_select,system_attrs_grid_id,system_attrs_search_space,state,data_name,estimator
84,16,1480.274362,2025-02-05 14:43:15.583819,2025-02-05 14:43:15.611825,0 days 00:00:00.028006,6,16,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_fs_3,LinearRegression


In [11]:
DIR_LOAD = 'train_poly/f_select_2025_02_05_14_43/'

X_train_poly_fs = utils_io.load_data(sub_dir= DIR_LOAD )

X_train_poly_fs['X_train_fs_1_LinearRegression'].shape, X_train_poly_fs['X_train_fs_2_LinearRegression'].shape, X_train_poly_fs['X_train_fs_3_LinearRegression'].shape

Файл data/train_poly/f_select_2025_02_05_14_43//X_train_fs_1_DecisionTreeRegressor.pkl успешно загружен.
Файл data/train_poly/f_select_2025_02_05_14_43//X_train_fs_1_LinearRegression.pkl успешно загружен.
Файл data/train_poly/f_select_2025_02_05_14_43//X_train_fs_2_DecisionTreeRegressor.pkl успешно загружен.
Файл data/train_poly/f_select_2025_02_05_14_43//X_train_fs_2_LinearRegression.pkl успешно загружен.
Файл data/train_poly/f_select_2025_02_05_14_43//X_train_fs_3_DecisionTreeRegressor.pkl успешно загружен.
Файл data/train_poly/f_select_2025_02_05_14_43//X_train_fs_3_LinearRegression.pkl успешно загружен.


((99, 6), (99, 8), (99, 6))

___
### <a id=5>SequentialFeatureSelector</a>

In [12]:
RAND_STATE = 1
scoring=['neg_root_mean_squared_error']
estimators_lst = [
    LinearRegression(),
    Ridge(alpha=0.2, random_state=RAND_STATE),
                ]

data_dict = {
    "X_train_sfs_1": (X_train_poly_1, target_train_1),
    "X_train_sfs_2": (X_train_poly_2, target_train_2),
    "X_train_sfs_3": (X_train_poly_3, target_train_3),
        }

___

In [13]:
result_df = feature_select.save_selected_features(data_dict=data_dict,
                                    selector=SequentialFeatureSelector,
                                    estimators=estimators_lst,
                                    scoring=scoring,
                                    output_dir=DIR_train,
                                    direction='backward')

result_df

Файл data/train_poly\f_select_2025_02_05_14_44\X_train_sfs_1_LinearRegression.pkl успешно сохранён.
Файл data/train_poly\f_select_2025_02_05_14_44\X_train_sfs_2_LinearRegression.pkl успешно сохранён.
Файл data/train_poly\f_select_2025_02_05_14_44\X_train_sfs_3_LinearRegression.pkl успешно сохранён.
Файл data/train_poly\f_select_2025_02_05_14_44\X_train_sfs_1_Ridge.pkl успешно сохранён.
Файл data/train_poly\f_select_2025_02_05_14_44\X_train_sfs_2_Ridge.pkl успешно сохранён.
Файл data/train_poly\f_select_2025_02_05_14_44\X_train_sfs_3_Ridge.pkl успешно сохранён.


,number,value,datetime_start,datetime_complete,duration,params_n_features_to_select,system_attrs_grid_id,system_attrs_search_space,state,data_name,estimator
0,0,12947.317264,2025-02-05 14:44:05.987830,2025-02-05 14:44:10.205838,0 days 00:00:04.218008,12,0,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_sfs_1,LinearRegression
1,1,12368.774618,2025-02-05 14:44:10.206836,2025-02-05 14:44:16.537840,0 days 00:00:06.331004,21,1,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_sfs_1,LinearRegression
2,2,30730.160756,2025-02-05 14:44:16.538839,2025-02-05 14:44:23.650336,0 days 00:00:07.111497,26,2,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_sfs_1,LinearRegression
3,3,12100.998212,2025-02-05 14:44:23.651342,2025-02-05 14:44:29.374342,0 days 00:00:05.723000,18,3,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_sfs_1,LinearRegression
4,4,12477.144174,2025-02-05 14:44:29.374342,2025-02-05 14:44:30.568345,0 days 00:00:01.194003,3,4,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_sfs_1,LinearRegression
...,...,...,...,...,...,...,...,...,...,...,...
199,29,1382.856087,2025-02-05 15:01:45.128443,2025-02-05 15:01:48.275443,0 days 00:00:03.147000,8,29,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_sfs_3,Ridge
200,30,6767.134652,2025-02-05 15:01:48.275443,2025-02-05 15:01:56.213436,0 days 00:00:07.937993,33,30,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_sfs_3,Ridge
201,31,3152.752311,2025-02-05 15:01:56.213436,2025-02-05 15:02:03.804437,0 days 00:00:07.591001,28,31,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_sfs_3,Ridge
202,32,1577.807831,2025-02-05 15:02:03.805436,2025-02-05 15:02:05.500443,0 days 00:00:01.695007,4,32,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_sfs_3,Ridge


In [15]:
min_result_df_1 = result_df[result_df['data_name'] == 'X_train_sfs_1' ]
min_result_df_1 = min_result_df_1[min_result_df_1['value'] == min_result_df_1['value'].min()]
min_result_df_1

,number,value,datetime_start,datetime_complete,duration,params_n_features_to_select,system_attrs_grid_id,system_attrs_search_space,state,data_name,estimator
27,27,12048.168675,2025-02-05 14:46:27.365477,2025-02-05 14:46:33.380476,0 days 00:00:06.014999,20,27,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_sfs_1,LinearRegression


In [16]:
min_result_df_2 = result_df[result_df['data_name'] == 'X_train_sfs_2' ]
min_result_df_2 = min_result_df_2[min_result_df_2['value'] == min_result_df_2['value'].min()]
min_result_df_2

,number,value,datetime_start,datetime_complete,duration,params_n_features_to_select,system_attrs_grid_id,system_attrs_search_space,state,data_name,estimator
55,21,1297.582123,2025-02-05 14:48:51.207613,2025-02-05 14:48:55.696614,0 days 00:00:04.489001,13,21,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_sfs_2,LinearRegression


In [17]:
min_result_df_3 = result_df[result_df['data_name'] == 'X_train_sfs_3' ]
min_result_df_3 = min_result_df_3[min_result_df_3['value'] == min_result_df_3['value'].min()]
min_result_df_3


,number,value,datetime_start,datetime_complete,duration,params_n_features_to_select,system_attrs_grid_id,system_attrs_search_space,state,data_name,estimator
178,8,1380.74456,2025-02-05 14:59:50.384174,2025-02-05 14:59:54.663176,0 days 00:00:04.279002,11,8,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_sfs_3,Ridge


In [20]:
DIR_LOAD = 'train_poly/f_select_2025_02_05_14_44/'

X_train_poly_fs = utils_io.load_data(sub_dir= DIR_LOAD )

X_train_poly_fs['X_train_sfs_1_LinearRegression'].shape, X_train_poly_fs['X_train_sfs_2_LinearRegression'].shape, X_train_poly_fs['X_train_sfs_3_Ridge'].shape

Файл data/train_poly/f_select_2025_02_05_14_44//X_train_sfs_1_LinearRegression.pkl успешно загружен.
Файл data/train_poly/f_select_2025_02_05_14_44//X_train_sfs_1_Ridge.pkl успешно загружен.
Файл data/train_poly/f_select_2025_02_05_14_44//X_train_sfs_2_LinearRegression.pkl успешно загружен.
Файл data/train_poly/f_select_2025_02_05_14_44//X_train_sfs_2_Ridge.pkl успешно загружен.
Файл data/train_poly/f_select_2025_02_05_14_44//X_train_sfs_3_LinearRegression.pkl успешно загружен.
Файл data/train_poly/f_select_2025_02_05_14_44//X_train_sfs_3_Ridge.pkl успешно загружен.


((99, 20), (99, 13), (99, 11))